# Install Dependencies

In [1]:
!pip install -q datasets

# Import Libraries

In [3]:
import torch
import datasets
from transformers import T5ForConditionalGeneration, T5Tokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

# Load Base model

In [ ]:
model_name = "google/mt5-small" 
tokenizer = T5Tokenizer.from_pretrained(model_name)  

config = T5ForConditionalGeneration.from_pretrained(model_name).config

model = T5ForConditionalGeneration.from_pretrained(model_name,config=config)

# Processing Dataset

## Split Dataset

In [ ]:
# Load the dataset
dataset_name = "OpenHust/vietnamese-summarization"
dataset = datasets.load_dataset(dataset_name)

# Define a function to preprocess the dataset
def preprocess_function(examples):
    inputs = examples["Document"]
    targets = examples["Summary"]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length", return_tensors="pt")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length", return_tensors="pt").input_ids
    labels[labels == tokenizer.pad_token_id] = -100  # Ignore padding in the loss
    model_inputs["labels"] = labels
    return model_inputs

# Apply the preprocessing function to the dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

# Split the dataset into training and validation sets
tokenized_datasets = tokenized_datasets['train'].train_test_split(test_size=0.1)

# Rename the splits to 'train' and 'val'
tokenized_datasets['val'] = tokenized_datasets.pop('test')


## Save Processed Dataset

In [ ]:
# Load the dataset from the local directory
local_dataset_dir = "dataset/raw_dataset"

# Save the dataset to the local directory
dataset.save_to_disk(local_dataset_dir)


## Save Processed Tokernizer

In [ ]:
import os

local_dataset_tokenizer_dir = "dataset/tokenizers"
os.makedirs(local_dataset_tokenizer_dir, exist_ok=True)

# Save the preprocessed dataset to the "dataset" folder
tokenized_datasets.save_to_disk(local_dataset_tokenizer_dir)

## Load Processed Dataset

In [ ]:
# Load the dataset from the local directory
dataset = dataset.load_from_disk("dataset/raw_dataset")

print("Dataset loaded from local directory")

## Load Processed Tokernizer

In [5]:
# Load the preprocessed dataset from the "dataset" folder
tokenized_datasets = datasets.load_from_disk("dataset/tokenizers")

# Training Process

## Define Hyperparameters

In [6]:
model_path = "viet-news-sum-mt5-small-finetune"
# Define the training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir=model_path,
    evaluation_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    num_train_epochs=10, # 40 -> 50
    weight_decay=0.01,
    save_total_limit=1,
    save_steps=500,
    logging_steps=100,
    predict_with_generate=True,
    # fp16=True if torch.cuda.is_available() else False,
    report_to="none" # Disable wandb reporting
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Train

In [7]:
import os
os.environ["WANDB_DISABLED"] = "true"

# Define the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['val'],
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

<ipython-input-7-1efba77810de>:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.078500,0.027335
2,0.074000,0.023692
3,0.066200,0.019857
4,0.060800,0.015207
5,0.053900,0.012891
6,0.052600,0.010073
7,0.046400,0.009463
8,0.046100,0.007866
9,0.044600,0.006860
10,0.052300,0.006372


TrainOutput(global_step=55930, training_loss=0.056790818105374, metrics={'train_runtime': 30781.5169, 'train_samples_per_second': 21.801, 'train_steps_per_second': 1.817, 'total_flos': 3.548280550588416e+17, 'train_loss': 0.056790818105374, 'epoch': 10.0})

# Save Finetune Model

In [9]:
# Save the model and tokenizer
model_path = "viet-news-sum-mt5-small-finetune"

# Save the tokenizer
tokenizer.save_pretrained(model_path)

# Save the model
model.save_pretrained(model_path)

print(f"Model and tokenizer saved to {model_path}")

Model and tokenizer saved to ./viet-sum-mt5-small-finetune


# Inference Process

In [ ]:
# Load the trained model and tokenizer
model_path = "viet-news-sum-mt5-small-finetune"
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

def preprocess_input(text):
    inputs = tokenizer(text, max_length=512, truncation=True, padding="max_length", return_tensors="pt")
    return inputs

# Define a function to generate the summary
def generate_summary(text):
    inputs = preprocess_input(text)
    with torch.no_grad():
        summary_ids = model.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=128, 
            early_stopping=True
        )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


In [ ]:
input_text = """
Vào ngày 8-1, khoa gây mê hồi sức Bệnh viện Đa khoa Đức Giang tiếp nhận bệnh nhân L.T.N.T. (23 tuổi, Chương Mỹ, Hà Nội) trong tình trạng hôn mê sau tai nạn giao thông.

Thai phụ mang thai 26 tuần bị viêm phổi, chấn thương sọ não nghiêm trọng với xuất huyết dưới nhện và tụ máu dưới màng cứng trán phải.

Theo bác sĩ Lê Nguyễn An - trưởng khoa gây mê hồi sức Bệnh viện Đa khoa Đức Giang, vấn đề thách thức trong quá trình điều trị với bệnh nhân này là việc cần phải đảm bảo sức khỏe cho cả mẹ và con là rất khó khăn.

"Các bác sĩ cố gắng duy trì tuổi thai ngoài 30 tuần để đảm bảo việc khi sinh ra trẻ có thể phát triển bình thường. Việc đảm bảo an toàn tính mạng cho mẹ cũng phải cân đối phù hợp, hạn chế tối thiểu việc ảnh hưởng tới thai nhi", bác sĩ An nói.

Trong suốt quá trình điều trị, các bác sĩ liên tục phối hợp với chuyên khoa sản và dinh dưỡng để đánh giá và điều chỉnh liên tục cho người bệnh để đảm bảo sự phát triển của em bé trong bụng mẹ.

Đặc biệt việc chăm sóc người bệnh ở trạng thái hôn mê, thở qua mở khí quản rất khó khăn, nhiều nguy cơ rủi ro về tình trạng nhiễm khuẩn, thiếu hụt dinh dưỡng, loét trợt điểm tì đè, nguy cơ suy thai".

Sau 70 ngày điều trị, tình trạng của sản phụ dần ổn định. Các chỉ số sinh tồn cải thiện, bệnh nhân tự thở qua mở khí quản, thai phát triển bình thường.

Tối 15-3, sản phụ có dấu hiệu chuyển dạ, thai 36 tuần (theo dự kiến sinh), ngôi ngược, ối vỡ sớm. Đội ngũ bác sĩ quyết định mổ lấy thai.

Ca phẫu thuật thành công, một bé trai nặng 2kg chào đời khóc to, niêm mạc hồng hào trong niềm hạnh phúc vô bờ của đội ngũ y bác sĩ và gia đình.

Ba ngày sau mổ, sản phụ tỉnh táo, tự ăn uống, được rút mở khí quản. Dự kiến cả mẹ và bé xuất viện trong ngày 21-3.
"""

input_text = input_text.replace("\n", "")

summary = generate_summary(input_text)
print(f"Summary: {summary}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Summary: Đến khoa gây mê hồi sức Bệnh viện Đức Giang , Hà Nội vừa tiếp nhận một nữ thai phụ mang thai 26 tuần bị viêm phổi , chấn thương sọ não nghiêm trọng với xuất huyết dưới nhện .
